# Import Libraries

In [1]:
import pandas as pd 
import numpy as np 
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

# Load Data

In [ ]:
data_path = '../data/'

In [3]:
# Import relevant tables
export_packstuecke_mva = pd.read_excel('../data/Export_Packstuecke_MVA.xlsx')
export_tbl_mva = pd.read_excel('../data/Export_TBL_MVA.xlsx')

In [4]:
# Merge relevant tables
data = pd.merge(export_tbl_mva, export_packstuecke_mva, left_on='ID', right_on='TBL_MVA_ID', how='outer')

# Data Cleaning and Translating

In [5]:
# Rename column names
data = data.rename(columns={'ID_x': 'id', 'ID_y': 'packstuecke_mva_id', 'TBL_MVA_ID': 'tbl_mva_id', 'PACKSTUECK_ID': 'package_id', 'LAENGE_IN_CM': 'length_in_cm', 'BREITE_IN_CM': 'width_in_cm', 'HOEHE_IN_CM': 'height_in_cm', 'GEWICHT_IN_KG': 'weight_in_kg', 'PACKSTUECKART': 'package_type', 'AUFTRAGSNUMMER': 'order_number', 'EINGANGSDATUM_UHRZEIT': 'receipt_date_time', 'LIEFERSCHEINNUMMER': 'delivery_receipt_number', 'STATUS': 'status', 'VERPACKT_DATUM_UHRZEIT': 'packed_date_time', 'SONDERFAHRT': 'special_trip', 'LIEFERSCHEIN_DATUM_UHRZEIT': 'delivery_number_date_time', 'BEREITGESTELLT_DATUM_UHRZEIT': 'provided_date_time', 'TA_DATUM_UHRZEIT': 'transport_order_date_time', 'NOTIZ': 'note', 'AUFTRAGANNAHME_DATUM_UHRZEIT': 'order_acceptance_date_time', 'DISPLAYNAME': 'display_name', 'LAND': 'country', 'GELOESCHT': 'deleted', 'BEMERKUNG_GELOESCHT': 'note_deleted', 'ARCHIV': 'archive', 'GELOESCHT_DATUM_UHRZEIT': 'deleted_date_time', 'TRACKING_NUMMER': 'tracking_number', 'DIENSTLEISTER': 'service_provider', 'KLAERUNG_LS': 'clarification_delivery_number', 'KLAERUNG_TA': 'clarification_transport_order', 'GEFAHRGUT': 'dangerous_goods', 'PO_NUMMER': 'purchase_order_number', 'ANGEKUENDIGT': 'announced', 'PRIO': 'priority', 'VERSENDER': 'sender'})

In [6]:
# Remove rows where "deleted" == 1
data = data.drop(data.loc[data['deleted'] == 1].index)

In [7]:
# Replace status from German to English
data['status'] = data['status'].replace('angeliefert', 'delivered')
data['status'] = data['status'].replace('verpackt', 'packed')
data['status'] = data['status'].replace('LS erstellt', 'delivery number created')
data['status'] = data['status'].replace('bereitgestellt', 'provided')
data['status'] = data['status'].replace('TA erstellt', 'transport order created')
data['status'] = data['status'].replace('in Klärung LS', 'in clarification delivery number')
data['status'] = data['status'].replace('in Klärung TA', 'in clarification transport order')
data['status'] = data['status'].replace('in Bearbeitung', 'in provision of the shipment')

In [8]:
# Replace package type from German to English
data['package_type'] = data['package_type'].replace('CAR', 'carton')
data['package_type'] = data['package_type'].replace('BEH', 'container')
data['package_type'] = data['package_type'].replace('PAL', 'pallet')
data['package_type'] = data['package_type'].replace('GBP', 'grid box')

In [9]:
# Clean country values
data['country'] = data['country'].replace('de', 'DE')
data['country'] = data['country'].replace('IN - Bidadi', 'IN')
data['country'] = data['country'].replace('DE - FCA Brucker', 'DE')
data['country'] = data['country'].replace('Tr', 'TR')
data['country'] = data['country'].replace('DE ', 'DE')
data['country'] = data['country'].replace(' DE', 'DE')
data['country'] = data['country'].replace(' AT ', 'AT')
data['country'] = data['country'].replace('^DE', 'DE')
data['country'] = data['country'].replace('DE - DAP | Schenker', 'DE')
data['country'] = data['country'].replace('Österreich', 'AT')
data['country'] = data['country'].replace('DE - Schenker/UPS', 'DE')
data['country'] = data['country'].replace('USA', 'US')
data['country'] = data['country'].replace('de- Koller', 'DE')
data['country'] = data['country'].replace('dE', 'DE')
data['country'] = data['country'].replace('De', 'DE')
data['country'] = data['country'].replace('D', 'DE')
data['country'] = data['country'].replace('at', 'AT')
data['country'] = data['country'].replace('AUT', 'AT')

# Remove rows where "country" has weird values
data = data.drop(data.loc[data['country'] == 'TNT | 070275454 | FCA'].index)
data = data.drop(data.loc[data['country'] == 'DER'].index)
data = data.drop(data.loc[data['country'] == 'FCA'].index)

In [10]:
# Filtering Rows

required_columns = ['id', 'special_trip', 'receipt_date_time', 'length_in_cm', 'width_in_cm', 'height_in_cm', 'weight_in_kg', 'package_type', 'packed_date_time', 'country', 'order_acceptance_date_time', 'delivery_number_date_time', 'provided_date_time', 'transport_order_date_time']
# Select rows with non-null values in the specified columns
data = data.dropna(subset=required_columns)

In [12]:
# Export table
data.to_excel('data.xlsx', index=False)

# Feaure Engineering

In [13]:
processed_data = data.copy()

In [14]:
# Feature extraction from timestamps

processed_data['receipt_date_time'] = pd.to_datetime(processed_data['receipt_date_time'])
processed_data['packed_date_time'] = pd.to_datetime(processed_data['packed_date_time'])
processed_data['order_acceptance_date_time'] = pd.to_datetime(processed_data['order_acceptance_date_time'])
processed_data['delivery_number_date_time'] = pd.to_datetime(processed_data['delivery_number_date_time'])
processed_data['provided_date_time'] = pd.to_datetime(processed_data['provided_date_time'])
processed_data['transport_order_date_time'] = pd.to_datetime(processed_data['transport_order_date_time'])

time_columns = ['receipt_date_time', 'packed_date_time', 'order_acceptance_date_time', 'delivery_number_date_time', 'provided_date_time']
name_mappings = {
    'receipt_date_time': 'receipt',
    'packed_date_time': 'packed',
    'order_acceptance_date_time': 'order_acceptance',
    'delivery_number_date_time': 'delivery_number',
    'provided_date_time': 'provided',
}
for col in time_columns:
    base_name = name_mappings[col]
    processed_data[f'{base_name}_day'] = processed_data[col].dt.day
    processed_data[f'{base_name}_month'] = processed_data[col].dt.month
    processed_data[f'{base_name}_year'] = processed_data[col].dt.year
    processed_data[f'{base_name}_hour'] = processed_data[col].dt.hour
    processed_data[f'{base_name}_minute'] = processed_data[col].dt.minute
    processed_data[f'{base_name}_second'] = processed_data[col].dt.second
    processed_data[f'{base_name}_weekday'] = processed_data[col].dt.weekday
    processed_data[f'{base_name}_time_of_day'] = pd.cut(
        processed_data[col].dt.hour,
        bins=[0, 6, 12, 18, 24],
        labels=['night', 'morning', 'afternoon', 'evening'],
        right=False
    )
    processed_data[f'{base_name}_week_of_year'] = processed_data[col].dt.isocalendar().week
    processed_data[f'{base_name}_is_weekend'] = (processed_data[col].dt.weekday >= 5).astype(int)
    processed_data[f'{base_name}_quarter'] = processed_data[col].dt.quarter
    processed_data[f'{base_name}_is_business_hour'] = processed_data[col].dt.hour.between(9, 17).astype(int)

processed_data['time_diff_receipt_to_packed'] = (processed_data['packed_date_time'] - processed_data['receipt_date_time']).dt.total_seconds()
processed_data['time_diff_packed_to_acceptance'] = (processed_data['order_acceptance_date_time'] - processed_data['packed_date_time']).dt.total_seconds()
processed_data['time_diff_acceptance_to_delivery'] = (processed_data['delivery_number_date_time'] - processed_data['order_acceptance_date_time']).dt.total_seconds()
processed_data['time_diff_delivery_to_provided'] = (processed_data['provided_date_time'] - processed_data['delivery_number_date_time']).dt.total_seconds()

In [15]:
# Feature extraction from length, width, height

processed_data['length_in_cm'] = processed_data['length_in_cm'].astype(float)
processed_data['width_in_cm'] = processed_data['width_in_cm'].astype(float)
processed_data['height_in_cm'] = processed_data['height_in_cm'].astype(float)
processed_data['weight_in_kg'] = processed_data['weight_in_kg'].astype(float)

# Calculate volume => Volume = length × width × height
processed_data['volume'] = processed_data['length_in_cm'] * processed_data['width_in_cm'] * processed_data['height_in_cm']

# Calculate density => Density = volume / weight
processed_data['density'] = processed_data['weight_in_kg'] / processed_data['volume']

In [16]:
# Select specific columns
selected_columns = ['id',
             'special_trip',
             'receipt_day',
             'receipt_month',
             'receipt_year',
             'receipt_hour',
             'receipt_minute',
             'receipt_second',
             'receipt_weekday',
             'receipt_week_of_year',
             'receipt_time_of_day',
             'receipt_is_weekend',
             'receipt_quarter',
             'receipt_is_business_hour',
             'receipt_date_time',
             'length_in_cm',
             'width_in_cm',
             'height_in_cm',
             'weight_in_kg',
             'volume',
             'density',
             'package_type',
             'time_diff_receipt_to_packed',
             'packed_day',
             'packed_month',
             'packed_year',
             'packed_hour',
             'packed_minute',
             'packed_second',
             'packed_weekday',
             'packed_time_of_day',
             'packed_week_of_year',
             'packed_is_weekend',
             'packed_quarter',
             'packed_is_business_hour',
             'packed_date_time',
             'country',
             'time_diff_packed_to_acceptance',
             'order_acceptance_day',
             'order_acceptance_month',
             'order_acceptance_year',
             'order_acceptance_hour',
             'order_acceptance_minute',
             'order_acceptance_second',
             'order_acceptance_weekday',
             'order_acceptance_time_of_day',
             'order_acceptance_week_of_year',
             'order_acceptance_is_weekend',
             'order_acceptance_quarter',
             'order_acceptance_is_business_hour',
             'order_acceptance_date_time',
             'time_diff_acceptance_to_delivery',
             'delivery_number_day',
             'delivery_number_month',
             'delivery_number_year',
             'delivery_number_hour',
             'delivery_number_minute',
             'delivery_number_second',
             'delivery_number_weekday',
             'delivery_number_time_of_day',
             'delivery_number_week_of_year',
             'delivery_number_is_weekend',
             'delivery_number_quarter',
             'delivery_number_is_business_hour',
             'delivery_number_date_time',
             'time_diff_delivery_to_provided',
             'provided_day',
             'provided_month',
             'provided_year',
             'provided_hour',
             'provided_minute',
             'provided_second',
             'provided_weekday',
             'provided_time_of_day',
             'provided_week_of_year',
             'provided_is_weekend',
             'provided_quarter',
             'provided_is_business_hour',
             'provided_date_time',
             'transport_order_date_time'
             ]

processed_data = processed_data[selected_columns]

In [17]:
# print all columns of data
pd.set_option('display.max_columns', None)

processed_data.head()

,id,special_trip,receipt_day,receipt_month,receipt_year,receipt_hour,receipt_minute,receipt_second,receipt_weekday,receipt_week_of_year,receipt_time_of_day,receipt_is_weekend,receipt_quarter,receipt_is_business_hour,receipt_date_time,length_in_cm,width_in_cm,height_in_cm,weight_in_kg,volume,density,package_type,time_diff_receipt_to_packed,packed_day,packed_month,packed_year,packed_hour,packed_minute,packed_second,packed_weekday,packed_time_of_day,packed_week_of_year,packed_is_weekend,packed_quarter,packed_is_business_hour,packed_date_time,country,time_diff_packed_to_acceptance,order_acceptance_day,order_acceptance_month,order_acceptance_year,order_acceptance_hour,order_acceptance_minute,order_acceptance_second,order_acceptance_weekday,order_acceptance_time_of_day,order_acceptance_week_of_year,order_acceptance_is_weekend,order_acceptance_quarter,order_acceptance_is_business_hour,order_acceptance_date_time,time_diff_acceptance_to_delivery,delivery_number_day,delivery_number_month,delivery_number_year,delivery_number_hour,delivery_number_minute,delivery_number_second,delivery_number_weekday,delivery_number_time_of_day,delivery_number_week_of_year,delivery_number_is_weekend,delivery_number_quarter,delivery_number_is_business_hour,delivery_number_date_time,time_diff_delivery_to_provided,provided_day,provided_month,provided_year,provided_hour,provided_minute,provided_second,provided_weekday,provided_time_of_day,provided_week_of_year,provided_is_weekend,provided_quarter,provided_is_business_hour,provided_date_time,transport_order_date_time
0,42,0,12,5,2023,4,57,57,4,19,night,0,2,0,2023-05-12 04:57:57,25.0,20.0,16.0,1.0,8000.0,0.000125,carton,26.0,12,5,2023,4,58,23,4,night,19,0,2,0,2023-05-12 04:58:23,DE,255824.0,15,5,2023,4,2,7,0,night,20,0,2,0,2023-05-15 04:02:07,500.0,15,5,2023,4,10,27,0,night,20,0,2,0,2023-05-15 04:10:27,91320.0,16,5,2023,5,32,27,1,night,20,0,2,0,2023-05-16 05:32:27,2023-05-16 09:20:43
1,43,0,12,5,2023,4,59,24,4,19,night,0,2,0,2023-05-12 04:59:24,25.0,20.0,16.0,2.0,8000.0,0.000250,carton,52.0,12,5,2023,5,0,16,4,night,19,0,2,0,2023-05-12 05:00:16,DE,256214.0,15,5,2023,4,10,30,0,night,20,0,2,0,2023-05-15 04:10:30,359.0,15,5,2023,4,16,29,0,night,20,0,2,0,2023-05-15 04:16:29,91096.0,16,5,2023,5,34,45,1,night,20,0,2,0,2023-05-16 05:34:45,2023-05-16 09:29:22
2,44,0,12,5,2023,5,2,54,4,19,night,0,2,0,2023-05-12 05:02:54,25.0,20.0,16.0,11.0,8000.0,0.001375,carton,17.0,12,5,2023,5,3,11,4,night,19,0,2,0,2023-05-12 05:03:11,DE,256400.0,15,5,2023,4,16,31,0,night,20,0,2,0,2023-05-15 04:16:31,315.0,15,5,2023,4,21,46,0,night,20,0,2,0,2023-05-15 04:21:46,90914.0,16,5,2023,5,37,0,1,night,20,0,2,0,2023-05-16 05:37:00,2023-05-16 09:37:26
3,45,0,12,5,2023,5,5,38,4,19,night,0,2,0,2023-05-12 05:05:38,37.0,28.0,16.0,2.0,16576.0,0.000121,carton,39.0,12,5,2023,5,6,17,4,night,19,0,2,0,2023-05-12 05:06:17,DE,256576.0,15,5,2023,4,22,33,0,night,20,0,2,0,2023-05-15 04:22:33,348.0,15,5,2023,4,28,21,0,night,20,0,2,0,2023-05-15 04:28:21,90677.0,16,5,2023,5,39,38,1,night,20,0,2,0,2023-05-16 05:39:38,2023-05-16 10:28:49
4,46,0,12,5,2023,5,8,49,4,19,night,0,2,0,2023-05-12 05:08:49,25.0,20.0,16.0,1.0,8000.0,0.000125,carton,69.0,12,5,2023,5,9,58,4,night,19,0,2,0,2023-05-12 05:09:58,DE,256706.0,15,5,2023,4,28,24,0,night,20,0,2,0,2023-05-15 04:28:24,329.0,15,5,2023,4,33,53,0,night,20,0,2,0,2023-05-15 04:33:53,90502.0,16,5,2023,5,42,15,1,night,20,0,2,0,2023-05-16 05:42:15,2023-05-16 10:28:52


In [18]:
# Get the first row to get idea of columns and values
first_row = processed_data.iloc[0]

# Create a dictionary of column-value pairs
column_value_pairs = {}
for column in processed_data.columns:
    column_value_pairs[column] = first_row[column]

# Print the column-value pairs
for column, value in column_value_pairs.items():
    print(f"{column}: {value}")


id: 42
special_trip: 0
receipt_day: 12
receipt_month: 5
receipt_year: 2023
receipt_hour: 4
receipt_minute: 57
receipt_second: 57
receipt_weekday: 4
receipt_week_of_year: 19
receipt_time_of_day: night
receipt_is_weekend: 0
receipt_quarter: 2
receipt_is_business_hour: 0
receipt_date_time: 2023-05-12 04:57:57
length_in_cm: 25.0
width_in_cm: 20.0
height_in_cm: 16.0
weight_in_kg: 1.0
volume: 8000.0
density: 0.000125
package_type: carton
time_diff_receipt_to_packed: 26.0
packed_day: 12
packed_month: 5
packed_year: 2023
packed_hour: 4
packed_minute: 58
packed_second: 23
packed_weekday: 4
packed_time_of_day: night
packed_week_of_year: 19
packed_is_weekend: 0
packed_quarter: 2
packed_is_business_hour: 0
packed_date_time: 2023-05-12 04:58:23
country: DE
time_diff_packed_to_acceptance: 255824.0
order_acceptance_day: 15
order_acceptance_month: 5
order_acceptance_year: 2023
order_acceptance_hour: 4
order_acceptance_minute: 2
order_acceptance_second: 7
order_acceptance_weekday: 0
order_acceptance_ti

In [19]:
# Export table
processed_data.to_excel(directory + 'processed_data.xlsx', index=False)